<font color=gray>Oracle Cloud Infrastructure Data Science Sample Notebook

Copyright (c) 2021 Oracle, Inc.  All rights reserved. <br>
Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>

# Uploading Larger Size Model Artifact Using Oracle ADS Library 

This notebook demonstrates simple solution for Oracle ADS Library which allows data scientists to upload larger model artifacts and eliminate the timeout error that is experienced by most folks when the artifact is large. It shows end-to-end steps from setting up the configuration till uploading the model artifact.

## Pre-requisites to Running this Notebook 

* We recommend that you run this notebook in a notebook session using a conda environment that has ADS version 2.3.1 installed
* You need access to the public internet

***
 
<font color=gray>Datasets are provided as a convenience. Datasets are considered Third Party Content and are not considered Materials under your agreement with Oracle applicable to the Services.
 
The dataset `oracle_classification_dataset1` is distributed under the [UPL license](oracle_data/UPL.txt). 
</font>
***

In [ ]:
import ads
import logging
import os
import tempfile
import warnings

from ads.catalog.model import ModelCatalog
from ads.common.model import ADSModel
from ads.common.model_export_util import prepare_generic_model
from ads.common.model_metadata import (MetadataCustomCategory,
                                       UseCaseType,
                                       Framework)
from ads.dataset.factory import DatasetFactory
from ads.feature_engineering.schema import Expression, Schema
from os import path
from sklearn.ensemble import RandomForestClassifier

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)
warnings.filterwarnings('ignore')
ads.set_documentation_mode(False)

In [ ]:
# Switch to 'api_key' if you prefer user principal (config + key file) as a method of authentication. 
# In this notebook, we are using resource principal. 
ads.set_auth(auth='resource_principal')

In [ ]:
# Load the dataset
ds_path = path.join("/", "opt", "notebooks", "ads-examples", "oracle_data", "oracle_classification_dataset1_150K.csv")

ds = DatasetFactory.open(ds_path, target="class")

ds
# Data preprocessing
transformed_ds = ds.auto_transform(fix_imbalance=False)
train, test = transformed_ds.train_test_split(test_size=0.15)

# Build the model and convert it to an ADSModel object
rf_clf = RandomForestClassifier(n_estimators=10).fit(train.X.values, train.y.values)
rf_model = ADSModel.from_estimator(rf_clf)

In [ ]:
# Prepare the model artifacts
path_to_ADS_model_artifact = tempfile.mkdtemp()

rf_model_artifact = rf_model.prepare(path_to_ADS_model_artifact, use_case_type=UseCaseType.BINARY_CLASSIFICATION,
                                     force_overwrite=True, data_sample=test, data_science_env=True,
                                     fn_artifact_files_included=False)

In [ ]:
# Saving the model artifact to the model catalog:
mc_model = rf_model_artifact.save(project_id=os.environ['PROJECT_OCID'],
                                  compartment_id=os.environ['NB_SESSION_COMPARTMENT_OCID'],
                                  training_id=os.environ['NB_SESSION_OCID'],
                                  display_name="<replace-with-your-display-name>",
                                  description="<replace-with-description>",
                                  ignore_pending_changes=True,
                                  timeout=1800,
                                  ignore_introspection=True,
                                 )
mc_model